In [3]:
from datasets import load_dataset
import pandas as pd

In [25]:
def sample_rows_from_dataset(dataset: str,
                             column_names: tuple,
                             *args,
                             num_samples: int = 100,
                             seed: int = 42,
                             **kwargs) -> pd.DataFrame:    
    if not isinstance(column_names, tuple):
        raise Exception("Column names need to be a list of column names as strings.")
    try:
        dataset = load_dataset(dataset, *args, split="test")
    except Exception as e:
        print("Could NOT load dataset for {0}".format(dataset))
        raise Exception("Error while loading dataset {}".format(e))
    shuffled_dataset = dataset.shuffle(seed=seed)
    df = pd.DataFrame(shuffled_dataset[:num_samples])
    try:
        return df[list(column_names)]
    except KeyError as e:
        raise e
    
    
def sample_rows_from_dataset(dataset: str,
                             column_names: tuple,
                             *args,
                             num_samples: int = 250,
                             seed: int = 42,
                             **kwargs) -> pd.DataFrame:
    if not isinstance(column_names, tuple):
        raise Exception("Column names need to be a list of column names as strings.")
    try:
        dataset = load_dataset(dataset, *args, **kwargs)
    except Exception as e:
        print("Could NOT load dataset for {0}".format(dataset))
        raise Exception("Error while loading dataset {}".format(e))
    shuffled_dataset = dataset.shuffle(seed=seed)
    df = pd.DataFrame(shuffled_dataset[:num_samples])
    try:
        return df[list(column_names)]
    except KeyError as e:
        raise e

### Squad Dataset

In [26]:
dataset_name = "squad"
configs = None
column_tuple = ("question", "context")

squad_qa_dataset = sample_rows_from_dataset(dataset_name, column_tuple, split="validation")

### Pubmed Biology Dataset

In [29]:
dataset_name = "pubmed_qa"
config = "pqa_labeled"
column_tuple = ("question", "context")

pubmed_qa_dataset = sample_rows_from_dataset(dataset_name, column_tuple, config, split="train")

contexts_strings = []

for i in range(len(pubmed_qa_dataset)):
    contexts_strings.append(' '.join(pubmed_qa_dataset["context"][i]['contexts']))
    
pubmed_qa_dataset['context'] = contexts_strings

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 4525.42 examples/s]


### BioASQ dataset

In [30]:
dataset_name = "BeIR/bioasq-generated-queries"
column_tuple = ("text", "query")

bioasq_qa_dataset = sample_rows_from_dataset(dataset_name, column_tuple, split="train")
bioasq_qa_dataset = bioasq_qa_dataset.rename(columns={"text": "context", "query": "question"})
bioasq_qa_dataset = bioasq_qa_dataset[["question", "context"]]

Extracting data files: 100%|██████████| 1/1 [01:16<00:00, 76.97s/it]
Generating train split: 14100000 examples [01:29, 158196.47 examples/s]


### cuad (legal) dataset

In [31]:
dataset_name = "cuad"
column_tuple = ("question", "context")

cuad_qa_dataset = sample_rows_from_dataset(dataset_name, column_tuple, split="train")

Generating test split: 100%|██████████| 4182/4182 [00:00<00:00, 15362.74 examples/s]


In [49]:
cuad_qa_dataset["domain"] = "legal"
bioasq_qa_dataset["domain"] = "bio"
pubmed_qa_dataset["domain"] = "bio"
squad_qa_dataset["domain"] = "None"

In [50]:
eval_dataset = pd.concat([cuad_qa_dataset, bioasq_qa_dataset, pubmed_qa_dataset, squad_qa_dataset], ignore_index=True)

In [52]:
eval_dataset.to_csv("eval_dataset.csv")